In [1]:
import os
import sys

device = "cuda:0"

# Setup: and Selenium chrome driver to PATH and set MINIWOB_BASE_URL env variable to the directory with HTML task files

In [2]:
from miniwob.environment import MiniWoBEnvironment
from miniwob.screenshot import pil_to_numpy_array

task_name = [
    'login-user',
    'email-inbox-forward',
    'email-inbox-delete',
    'email-inbox-important'
]
num_task = len(task_name) if type(task_name) is list else 1
base_url = os.environ.get('MINIWOB_BASE_URL')
print('BASE URL:', base_url)

BASE URL: file:///mnt/akostin/home/akostin/prjs/miniwob-plusplus/html/


In [3]:
NH=128
X_MAX, Y_MAX = 160, 210
DOM_TEXT_MAX_TOKENS = 8
DOM_CLASSES_MAX_TOKENS = 8
GOAL_MAX_TOKENS = 32
GOAL_MAX_WORDS = 20
GOAL_WORD_MAX_TOKENS = 8
DOM_MAX_EDGES = 7
DOM_MAX_VERTICES = 17
EXPLORE_REWARD = 0.05
STEP_PRICE = 0.1
HEADS_KEYS = ["dom", "token", "master"]
H_bert = 64

# Gym miniwob wrapper

In [4]:
import gym
import torch
from miniwob.action import MiniWoBElementClick, MiniWoBType
from transformers import BertTokenizer
import numpy as np


class MiniWobClickElementWrapper(gym.Wrapper):
    
    def __init__(self, env):
        super().__init__(env)
        self.env = env
        bert_model = 'bert-base-cased'
        
        self.tokenizer = BertTokenizer.from_pretrained(bert_model)
        self.observation_space = gym.spaces.Dict({"goal_state": gym.spaces.Box(0, 
                                                                               self.tokenizer.vocab_size,
                                                                               (GOAL_MAX_TOKENS, ),
                                                                               dtype=np.int64),
                                                  #"img_state": gym.spaces.Box(0,
                                                  #                            255,
                                                  #                            (150, 150, 3),
                                                  #                            dtype=np.uint8),
                                                  "dom_simple_feats": gym.spaces.Box(-1,
                                                                                     1,
                                                                                     (DOM_MAX_VERTICES, 3),
                                                                                     dtype=np.float32),
                                                  "dom_text_tokens": gym.spaces.Box(0,
                                                                                    self.tokenizer.vocab_size,
                                                                                    (DOM_MAX_VERTICES, DOM_TEXT_MAX_TOKENS),
                                                                                    dtype=np.int64),
                                                  "dom_classes_tokens": gym.spaces.Box(0,
                                                                                       self.tokenizer.vocab_size,
                                                                                       (DOM_MAX_VERTICES, DOM_CLASSES_MAX_TOKENS),
                                                                                       dtype=np.int64),
                                                  #"dom_tags": gym.spaces.Box(0,
                                                  #                           len(tags)-1,
                                                  #                           (DOM_MAX_VERTICES,),
                                                  #                           dtype=np.int64),
                                                  #"n_goal_words": gym.spaces.Box(0,
                                                  #                               GOAL_MAX_WORDS,
                                                  #                               (1,),
                                                  #                               dtype=np.int64),
                                                  "action_mask": gym.spaces.Box(0,
                                                                                1,
                                                                                (DOM_MAX_VERTICES + GOAL_MAX_WORDS,),
                                                                                dtype=np.int64),
                                                  "goal_words": gym.spaces.Box(0,
                                                                               self.tokenizer.vocab_size,
                                                                               (GOAL_MAX_WORDS, GOAL_WORD_MAX_TOKENS),
                                                                               dtype=np.int64)})
        
        self.n_steps = [0 for _ in range(self.num_instances)]
        self.past_states = [list() for _ in range(self.num_instances)]
        self.curr_states = [torch.zeros((DOM_MAX_VERTICES, 2)) for _ in range(self.num_instances)]
        self.inst_verts = [None for _ in range(self.num_instances)]
        self.text_tokens = [None for _ in range(self.num_instances)]
        self.dom_keys = [key for key in self.observation_space if key.startswith("dom")] + ["action_mask"]
        self.action_space = gym.spaces.Discrete(DOM_MAX_VERTICES + GOAL_MAX_WORDS)
    
    def dom_filter(self, ob):
        f_elems = list()
        elems = ob.dom_elements
        for e in elems:
            if ((not e.ref is None) and e.is_leaf):
                f_elems.append(e)
        return f_elems[:DOM_MAX_VERTICES]
    
    def clean_classes(self, text):
        return text.replace(' ', '; ').replace('-', ' ').replace('_', ' ')
    
    def _ob_to_dom(self, ob, i):
        max_length = DOM_TEXT_MAX_TOKENS
        if ob is None:
            self.inst_verts[i] = None
            self.text_tokens[i] = None
            return {
                "dom_simple_feats": -torch.ones((1, DOM_MAX_VERTICES, 3), dtype=torch.float32),
                "dom_text_tokens": torch.zeros((1, DOM_MAX_VERTICES, DOM_TEXT_MAX_TOKENS), dtype=torch.int64),
                "dom_classes_tokens": torch.zeros((1, DOM_MAX_VERTICES, DOM_CLASSES_MAX_TOKENS), dtype=torch.int64),
                #"dom_tags": torch.zeros((1, DOM_MAX_VERTICES), dtype=torch.int64),
                #"n_goal_words": torch.zeros((1,1), dtype=torch.int64),
                "action_mask": torch.ones((1, DOM_MAX_VERTICES + GOAL_MAX_WORDS), dtype=torch.int64)
            }
        else:
            self.inst_verts[i] = self.dom_filter(ob)
            self.text_tokens[i] = ob.tokens
            tokens = []
            class_tokens = []
            feats = []
            tags_ = []
            available_dom_click = []
            for elem in self.dom_filter(ob):
                elem_text = elem.text if elem.text else "none"
                elem_tokens = self.tokenizer(elem_text,
                                             padding='max_length',
                                             max_length=DOM_TEXT_MAX_TOKENS,
                                             truncation=True,
                                             return_tensors="pt",
                                             add_special_tokens=False)["input_ids"]
                tokens.append(elem_tokens)
                
                elem_text = self.clean_classes(elem.classes if elem.classes else "none")
                elem_tokens = self.tokenizer(elem_text,
                                             padding='max_length',
                                             max_length=DOM_CLASSES_MAX_TOKENS,
                                             truncation=True,
                                             return_tensors="pt",
                                             add_special_tokens=False)["input_ids"]
                class_tokens.append(elem_tokens)
                
                elem_feats = torch.tensor([elem.focused, bool(elem.value), elem.tampered],
                                          dtype=torch.float32)
                feats.append(elem_feats)
                #tags_.append(torch.tensor([tag2id[elem.tag]], dtype=torch.int64))
                available_dom_click.append(0 if elem.is_leaf else 1)
            
            delta_v = DOM_MAX_VERTICES - len(tokens)
            available_dom_click += [1 for _ in range(delta_v)]
            
            if self.typeble_elem_focused(i) and not self.elem_have_value(i):
                available_type_tokens = [0 for _ in range(len(self.text_tokens[i]))] + \
                                        [1 for _ in range(GOAL_MAX_WORDS - len(self.text_tokens[i]))]
            else:
                #print("typing not available")
                available_type_tokens = [1 for _ in range(GOAL_MAX_WORDS)]
            available_actions = available_dom_click + available_type_tokens
            
            tokens += [torch.zeros((1, DOM_TEXT_MAX_TOKENS), dtype=torch.int64) for _ in range(delta_v)]
            class_tokens += [torch.zeros((1, DOM_CLASSES_MAX_TOKENS), dtype=torch.int64) for _ in range(delta_v)]
            feats += [-torch.ones((3,), dtype=torch.float32) for _ in range(delta_v)]
            tags_ += [torch.zeros((1,), dtype=torch.int64) for _ in range(delta_v)]
            
            return{
                "dom_text_tokens": torch.cat(tokens, dim=0).unsqueeze(0),
                "dom_classes_tokens": torch.cat(class_tokens, dim=0).unsqueeze(0),
                "dom_simple_feats": torch.stack(feats, dim=0).unsqueeze(0),
                #"dom_tags": torch.cat(tags_, dim=0).unsqueeze(0),
                #"n_goal_words": torch.tensor([len(self.text_tokens[i])]).unsqueeze(0),
                "action_mask": torch.tensor(available_actions).unsqueeze(0)
            }
    
    def typeble_elem_focused(self, i):
        focused_elem = self.n_element_focused(i)
        if not focused_elem is None:
            focused_vert = self.inst_verts[i][focused_elem]
            return ") value=" in str(focused_vert) and "input" in str(focused_vert)
        else:
            return False
    
    def elem_have_value(self, i):
        focused_elem = self.n_element_focused(i)
        if not focused_elem is None:
            value_state = self.curr_states[i][focused_elem, 1]
            return bool(value_state)
        else:
            return False
    
    def n_element_focused(self, i):
        focus_state = self.curr_states[i][:, 0]
        focus = torch.where(focus_state==1)[0]
        if len(focus)>0:
            return int(focus[0])
        else:
            return None
    
    def _ob_to_token(self, ob, i):
        max_length = GOAL_MAX_TOKENS
        if ob is None:
            return torch.zeros(1, max_length)
        else:
            return self.tokenizer(ob.tokens,  
                                  padding='max_length', 
                                  max_length = max_length, truncation=True,          
                                  return_tensors="pt",
                                  is_split_into_words=True)['input_ids']
    
    def _ob_to_goal_words(self, ob):
        max_length = GOAL_WORD_MAX_TOKENS
        if ob is None:
            return torch.zeros(1, GOAL_MAX_WORDS, GOAL_WORD_MAX_TOKENS)
        else:
            words = list()
            for t in ob.tokens:
                word_tokens = self.tokenizer(t,  
                                             padding='max_length', 
                                             max_length = max_length, truncation=True,          
                                             return_tensors="pt",
                                             is_split_into_words=False,
                                             add_special_tokens=False)['input_ids']
                words.append(word_tokens)
            delta = GOAL_MAX_WORDS - len(words)
            words += [torch.zeros(1, max_length) for _ in range(delta)]
            words = torch.cat(words, dim=0).unsqueeze(0)
            return words
            
    def _ob_to_image(self, ob):
        if ob is None:
            return torch.zeros(1, 150, 150, 3)
        else:
            return torch.tensor(pil_to_numpy_array(ob.screenshot.resize([150, 150]))).unsqueeze(0)
        
    
    def _to_miniwob_actions(self, actions):
        
        #actions = actions.squeeze(dim=1)
        miniwob_actions = []
        
        for i in range(self.num_instances):
            if self.instances[i].get_metadata()['done']:
                miniwob_actions.append(None)
            else:
                if self.inst_verts[i] is None:
                    miniwob_actions.append(None)
                else:
                    v_n = actions[i]#.item()
                    if v_n is None:
                        miniwob_actions.append(v_n)
                    elif v_n < DOM_MAX_VERTICES:
                        element = self.inst_verts[i][v_n]
                        miniwob_actions.append(MiniWoBElementClick(element))
                        self.n_steps[i] += 1
                    else:
                        text = self.text_tokens[i][v_n - DOM_MAX_VERTICES]
                        miniwob_actions.append(MiniWoBType(text))
                        self.n_steps[i] += 1
        return miniwob_actions
    
    def get_explore_reward(self, obs):
        explore_rewards = [0 for _ in range(self.num_instances)]
        for i, states in enumerate(self.past_states):
            t = obs["dom_simple_feats"][i][:, :-1]
            unique = True
            for pt in states:
                if torch.all(t==pt):
                    unique = False
                    break
            if unique:
                explore_rewards[i] += EXPLORE_REWARD
            else:
                explore_rewards[i] -= EXPLORE_REWARD
        return explore_rewards

    def observation(self, obs):
        goals = torch.cat([self._ob_to_token(obs[i], i) for i in range(len(obs))], dim=0)
        goal_words = torch.cat([self._ob_to_goal_words(ob) for ob in obs], dim=0)
        #imgs = torch.cat([self._ob_to_image(ob) for ob in obs], dim=0)
        doms = [self._ob_to_dom(obs[i], i) for i in range(len(obs))]
        doms_states = {key: torch.cat([dom[key] for dom in doms], dim=0) for key in self.dom_keys}
        return {#'img_state': imgs, 
                'goal_state': goals,
                "goal_words": goal_words,
                **doms_states}
    
    def _ob_to_simple_feats(self, ob):
        if ob is None:
            return -torch.ones((1, DOM_MAX_VERTICES, 3), dtype=torch.float32)
        else:
            feats = []
            for elem in self.dom_filter(ob):
                elem_feats = torch.tensor([elem.focused, bool(elem.value), elem.tampered],
                                          dtype=torch.float32)
                feats.append(elem_feats)            
            delta_v = DOM_MAX_VERTICES - len(feats)
            feats += [-torch.ones((3,), dtype=torch.float32) for _ in range(delta_v)]
            return torch.stack(feats, dim=0).unsqueeze(0)
    
    def get_rewards(self, rewards):
        augmented_rewards = list()
        for i, reward in enumerate(rewards):
            if reward > 0:
                if self.n_steps[i] < 5:
                    r = 1 - self.n_steps[i] * 0.1
                else:
                    r = 0.5 * (np.e ** (-self.n_steps[i] + 5))
                augmented_rewards.append(r)
            else:
                augmented_rewards.append(reward)
        return augmented_rewards
        
    def update_curr_states(self, obs):
        obs = [self._ob_to_simple_feats(ob) for ob in obs]
        obs = {"dom_simple_feats": torch.cat(obs, dim=0)}
        for i, _ in enumerate(self.past_states):
            s = obs["dom_simple_feats"][i][:, :-1]
            self.curr_states[i] = s
    
    def update_past_states(self, obs):
        for i, _ in enumerate(self.past_states):
            s = obs["dom_simple_feats"][i][:, :-1]
            self.past_states[i].append(s)
    
    def reset(self):
        obs = self.observation(self.env.reset(record_screenshots=True))
        self.update_past_states(obs)
        return obs
    
    def step(self, actions):
        miniwob_actions = self._to_miniwob_actions(actions)
        obs, rewards, dones, infos = self.env.step(miniwob_actions)
        self.update_curr_states(obs)

        obs = self.observation(obs)
        
        ###
        rewards = self.get_rewards(rewards)
        ###
        
        ####
        explore_rewards = self.get_explore_reward(obs)
        rewards = [a + b for a, b in zip(rewards, explore_rewards)]
        for i, r in enumerate(explore_rewards):
            infos['n'][i]['exploration_reward'] = r
        ####
        
        for i, instance in enumerate(self.env.instances):
            if instance.get_metadata()['done']:
                instance.begin_task()
                self.past_states[i] = list()
                self.n_steps[i] = 0
        
        self.update_past_states(obs)
        
        return obs, torch.tensor([rewards]).T, torch.tensor([dones]).T, infos['n']


class EpisodeInfoWrapper(gym.Wrapper):
    
    def __init__(self, env, n):
        super(EpisodeInfoWrapper, self).__init__(env)
        self.episode_reward = np.zeros(n)
        self.episode_steps = np.zeros(n)
        self.n = n

    def reset(self):
        self.episode_reward = np.zeros(self.n)
        self.episode_steps = np.zeros(self.n)
        return self.env.reset()

    def step(self, actions):
        states, rewards, dones, infos = self.env.step(actions)
        
        for i, done in enumerate(dones):
            self.episode_steps[i] += 1
            if done:
                self.episode_reward[i] = rewards[i]
                infos[i]['episode'] = {'r': self.episode_reward[i], 'steps': self.episode_steps[i]}
                self.episode_steps[i] = 0
                
        return states, rewards, dones, infos

In [5]:
from transformers import BertTokenizer, BertModel, BertConfig
import torch
from torch import nn
from torch.nn import Linear
from torch.nn.functional import relu


class BertEncoder(nn.Module):
    
    def __init__(self):
        
        super().__init__()
        bert_model = 'bert-base-cased'
        
        self.bert_encoder = BertModel.from_pretrained(bert_model).embeddings.word_embeddings
        self.embed_dim = 768
        self.output_size = H_bert
        self.h_dim = 128
        self.n_h_layers = 2
        
        self.out_layers = nn.ModuleList(
            [nn.Sequential(nn.LayerNorm(self.embed_dim), nn.Linear(self.embed_dim, self.h_dim), nn.ReLU())] +
            [nn.Sequential(nn.LayerNorm(self.h_dim), nn.Linear(self.h_dim, self.h_dim), nn.ReLU()) for _ in range(self.n_h_layers)] +
            [nn.Sequential(nn.LayerNorm(self.h_dim), nn.Linear(self.h_dim, self.output_size), nn.ReLU())]
        )
           
    def forward(self, input_ids):
        with torch.no_grad():
            text_tokens_embed = self.bert_encoder(input_ids.long()).mean(dim=1)
        
        prev, out = text_tokens_embed, text_tokens_embed
        for l in self.out_layers:
            out = l(out)
            if out.shape[-1] == prev.shape[-1]:
                out = out + prev
            prev = out
        
        return out

In [6]:
import torch
from torch import nn
from torch.nn import ModuleList, Parameter, TransformerEncoderLayer, TransformerEncoder
from torch.nn.functional import relu, softmax


UNDIRECTED = True
SELF_LOOP = True
bert_model = 'bert-base-cased'
tokenizer = BertTokenizer.from_pretrained(bert_model)
pad_token_id = tokenizer.pad_token_id


class FFNet(nn.Module):
    def __init__(self, inp_dim, h_dim=64, out_dim=64, h_layers=4, last_layer_relu=True):
        super(FFNet, self).__init__()
        self.inp_dim = inp_dim
        self.h_dim = h_dim
        self.out_dim = out_dim
        
        layers = []
        layers.append(nn.Sequential(nn.LayerNorm(self.inp_dim), nn.Linear(self.inp_dim, self.h_dim), nn.ReLU()))
        for i in range(h_layers):
            layers.append(nn.Sequential(nn.LayerNorm(self.h_dim), nn.Linear(self.h_dim, self.h_dim), nn.ReLU()))
        
        if last_layer_relu:
            layers.append(nn.Sequential(nn.LayerNorm(self.h_dim), nn.Linear(self.h_dim, self.out_dim), nn.ReLU()))
        else:
            layers.append(nn.Sequential(nn.LayerNorm(self.h_dim), nn.Linear(self.h_dim, self.out_dim)))
        
        self.layers = nn.ModuleList(layers)
    
    def forward(self, input):
        out = input
        prev = input
        for l in self.layers:
            out = l(out)
            if out.shape[-1] == prev.shape[-1]:
                out = out + prev
            prev = out
        return out


class Encoder(nn.Module):
    def __init__(self):
        super(Encoder, self).__init__()
        self.text_encoder = BertEncoder()
        self.tf_size = 64
        self.output_size = 128
        
        self.net = FFNet(((H_bert * 2 + 3) * DOM_MAX_VERTICES + GOAL_MAX_WORDS * H_bert), 512, self.output_size, 8)
        
        self.flag = True
        self.buffer = None
    
    def forward(self, input):
        if self.flag:
            bs = input["goal_state"].shape[0]
            
            # goal words encoding
            goal_words = torch.flatten(input["goal_words"], end_dim=-2)
            words = self.text_encoder(goal_words).view(bs, -1)
            
            # dom elements encoding
            dom_text = torch.flatten(input["dom_text_tokens"], end_dim=-2)
            dom_text_global = self.text_encoder(dom_text)
            
            dom_classes = torch.flatten(input["dom_classes_tokens"], end_dim=-2)
            dom_classes_global = self.text_encoder(dom_classes)
            
            simple_feats = torch.flatten(input["dom_simple_feats"], end_dim=-2)
            
            doms = torch.cat([dom_text_global, dom_classes_global, simple_feats], dim=-1)
            doms = doms.view(bs, -1)
            
            # forward
            inp = torch.cat([doms, words], dim=-1)#.view(bs, -1)
            out = self.net(inp)
            
            self.buffer = out
        
        else:
            out = self.buffer
        
        return out

In [7]:
from rllr.models.ppo import FixedCategorical, ActorCriticNetwork, CriticNetwork
from torch.nn import functional as F


class DiscreteActorNetwork(nn.Module):
    """
    Actor is a policy network. Given state it evaluates
    probability of action given state or sample an action
    """

    def __init__(self, action_size, state_encoder, hidden_size):
        super().__init__()
        self.state_encoder = state_encoder
        input_size = state_encoder.output_size
        self.h_dim = hidden_size
        self.inp_dim = input_size
        self.output_size = action_size
        self.dom_head_out = DOM_MAX_VERTICES
        self.token_head_out = GOAL_MAX_WORDS
        self.master_head_out = 2
        self.n_h_layers = 4
        
        # dom layers
        self.dom_layers = ModuleList([nn.Sequential(nn.LayerNorm(self.inp_dim), nn.Linear(self.inp_dim, self.h_dim), nn.ReLU())] + \
                                     [nn.Sequential(nn.LayerNorm(self.h_dim), nn.Linear(self.h_dim, self.h_dim), nn.ReLU()) for _ in range(self.n_h_layers)] + \
                                     [nn.Sequential(nn.LayerNorm(self.h_dim), nn.Linear(self.h_dim, self.dom_head_out))])
        
        # token layers
        self.token_layers = ModuleList([nn.Sequential(nn.LayerNorm(self.inp_dim), nn.Linear(self.inp_dim, self.h_dim), nn.ReLU())] + \
                                       [nn.Sequential(nn.LayerNorm(self.h_dim), nn.Linear(self.h_dim, self.h_dim), nn.ReLU()) for _ in range(self.n_h_layers)] + \
                                       [nn.Sequential(nn.LayerNorm(self.h_dim), nn.Linear(self.h_dim, self.token_head_out))])
        
        # layers for master head
        self.master_layers = ModuleList([nn.Sequential(nn.LayerNorm(self.inp_dim), nn.Linear(self.inp_dim, self.h_dim), nn.ReLU())] + \
                                        [nn.Sequential(nn.LayerNorm(self.h_dim), nn.Linear(self.h_dim, self.h_dim), nn.ReLU()) for _ in range(self.n_h_layers)] + \
                                        [nn.Sequential(nn.LayerNorm(self.h_dim), nn.Linear(self.h_dim, self.master_head_out))])
        
    def forward(self, states):
        states_encoding = self.state_encoder(states)
        action_mask = states["action_mask"]  # BxA
        
        dom_mask, token_mask = action_mask[:, :DOM_MAX_VERTICES], action_mask[:, -GOAL_MAX_WORDS:]
        master_mask = torch.cat([torch.all(dom_mask, dim=1, keepdim=True),
                                 torch.all(token_mask, dim=1, keepdim=True)],
                                 dim=1).int()
        for i in range(token_mask.shape[0]):
            if dom_mask[i].all():
                dom_mask[i] = torch.zeros_like(dom_mask[i], dtype=torch.int)
            if token_mask[i].all():
                token_mask[i] = torch.zeros_like(token_mask[i], dtype=torch.int)
            if master_mask[i].all():
                master_mask[i] = torch.zeros_like(master_mask[i], dtype=torch.int)
        
        heads_mask = {k: v for k, v in zip(HEADS_KEYS, [dom_mask, token_mask, master_mask])}
        
        heads_out = list()
        for layers in [self.dom_layers, self.token_layers, self.master_layers]:
            out = states_encoding
            prev = states_encoding
            for l in layers:
                out = l(out)
                if out.shape[-1] == prev.shape[-1]:
                    out = out + prev
                prev = out
            heads_out.append(out)
        
        heads_out = {k: v for k, v in zip(HEADS_KEYS, heads_out)}
        
        for k in heads_mask:
            heads_out[k] = torch.where(heads_mask[k]==1, -float("inf"), heads_out[k].double())
        
        return {k: FixedCategorical(logits=F.log_softmax(v, dim=1)) for k, v in heads_out.items()}
    


class ACN(ActorCriticNetwork):
    def __init__(self, action_space, actor_state_encoder, critic_state_encoder, actor_hidden_size, critic_hidden_size):
        super(ActorCriticNetwork, self).__init__()
        if type(action_space) == gym.spaces.Discrete:
            self.actor = DiscreteActorNetwork(action_space.n, actor_state_encoder, actor_hidden_size)
        else:
            raise f'{action_space} not supported'
        self.critic = CriticNetwork(critic_state_encoder, critic_hidden_size)

        def init_params(m):
            classname = m.__class__.__name__
            if classname.find("Linear") != -1:
                m.weight.data.normal_(0, 1)
                m.weight.data *= 1 / torch.sqrt(m.weight.data.pow(2).sum(1, keepdim=True))
                if m.bias is not None:
                    m.bias.data.fill_(0)

        self.apply(init_params)
    
    def act(self, states, deterministic=False):
        self.actor.state_encoder.flag = True
        dist = self.actor.forward(states)
        if deterministic:
            action = {k: dist[k].mode() for k in dist}
        else:
            action = {k: dist[k].sample() for k in dist}
        log_probs = {k: dist[k].log_probs(action[k]) for k in dist}
        self.critic.state_encoder.flag = False
        return self.critic.forward(states), action, log_probs
    
    def evaluate_actions(self, states, actions):
        self.actor.state_encoder.flag = True
        dist = self.actor.forward(states)
        self.critic.state_encoder.flag = False
        values = self.critic.forward(states)
        
        log_probs = {k: dist[k].log_probs(actions[k]) for k in dist}
        dist_entropy = {k: dist[k].entropy().mean() for k in dist}
        return values, log_probs, dist_entropy

# Train agent

In [8]:
import numpy as np
from rllr.env.vec_wrappers import make_vec_envs
from stable_baselines3.common.vec_env import DummyVecEnv


def get_envs(n, **kwargs):
    envs = list()
    for task in task_name:
        env = MiniWoBEnvironment(task, seeds=range(n), num_instances=n, base_url=base_url, **kwargs)
        env = MiniWobClickElementWrapper(env)
        env = EpisodeInfoWrapper(env, n)
        envs.append(env)
    return envs

In [9]:
n_instances = 4
envs = get_envs(n_instances, headless=True)

In [10]:
from rllr.models.encoders import GoalStateEncoder, SimpleCNN

encoder = Encoder()

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [11]:
from rllr.algo.ppo import PPO

class MultiheadPPO(PPO):
    def __init__(self, actor_critic, clip_param, ppo_epoch, num_mini_batch, value_loss_coef,
                 entropy_coef, lr=None, eps=None, max_grad_norm=None,
                 dom_entropy=1, token_entropy=1, master_entropy=1):
        super(MultiheadPPO, self).__init__(actor_critic, clip_param, ppo_epoch,
                                           num_mini_batch, value_loss_coef,
                                           entropy_coef, lr, eps, max_grad_norm)
        self.dom_entropy = dom_entropy
        self.token_entropy = token_entropy
        self.master_entropy = master_entropy
    
    def update(self, rollouts):
        advantages = rollouts.returns[:-1] - rollouts.value_preds[:-1]
        advantages = (advantages - advantages.mean()) / (advantages.std() + 1e-5)

        value_loss_epoch = 0
        action_loss_epoch = 0
        dist_entropy_epoch = 0

        for e in range(self.ppo_epoch):
            data_generator = rollouts.feed_forward_generator(advantages, self.num_mini_batch)

            for sample in data_generator:
                obs_batch, actions_batch, value_preds_batch, return_batch, masks_batch, \
                old_action_log_probs_batch, adv_targ = sample

                # Reshape to do in a single forward pass for all steps
                values, action_log_probs, dist_entropy = self.actor_critic.evaluate_actions(obs_batch, actions_batch)

                ratio = {k: torch.exp(action_log_probs[k] - old_action_log_probs_batch[k]) for k in action_log_probs}
                surr1 = {k: ratio[k] * adv_targ for k in ratio}
                surr2 = {k: torch.clamp(ratio[k], 1.0 - self.clip_param, 1.0 + self.clip_param) * adv_targ for k in ratio}
                action_loss = {k: -torch.min(surr1[k], surr2[k]) for k in surr1}

                # clipped_value_loss:
                value_pred_clipped = value_preds_batch + \
                     (values - value_preds_batch).clamp(-self.clip_param, self.clip_param)
                value_losses = (values - return_batch).pow(2)
                value_losses_clipped = (value_pred_clipped - return_batch).pow(2)
                value_loss = 0.5 * torch.max(value_losses, value_losses_clipped).mean()

                self.optimizer.zero_grad()
                
                action_loss = action_loss["master"] + torch.where(actions_batch["master"]==0, action_loss["dom"], action_loss["token"])
                action_loss = action_loss.mean()
                entropy_loss = dist_entropy["master"] * self.master_entropy + \
                               dist_entropy["dom"] * self.dom_entropy + \
                               dist_entropy["token"] * self.token_entropy
                loss = value_loss * self.value_loss_coef + action_loss - entropy_loss * self.entropy_coef
                
                loss.backward()
                nn.utils.clip_grad_norm_(self.actor_critic.parameters(), self.max_grad_norm)
                self.optimizer.step()

                value_loss_epoch += value_loss.item()
                action_loss_epoch += action_loss.item()
                dist_entropy_epoch += entropy_loss.item()

        num_updates = self.ppo_epoch * self.num_mini_batch

        value_loss_epoch /= num_updates
        action_loss_epoch /= num_updates
        dist_entropy_epoch /= num_updates

        return value_loss_epoch, action_loss_epoch, dist_entropy_epoch


In [12]:
from rllr.buffer.rollout import RolloutStorage
from torch.utils.data.sampler import BatchSampler, SubsetRandomSampler


class MultiheadRolloutStorage(RolloutStorage):
    def __init__(self, num_steps, num_processes, obs_space, action_space):
        super(MultiheadRolloutStorage, self).__init__(num_steps, num_processes, obs_space, action_space)
        if action_space.__class__.__name__ == 'Discrete':
            self.actions = {k: torch.zeros(num_steps, num_processes, 1).long() for k in HEADS_KEYS}
        else:
            self.actions = {k: torch.zeros(num_steps, num_processes, action_space.shape[0]) for k in HEADS_KEYS}
        self.action_log_probs = {k: torch.zeros(num_steps, num_processes, 1) for k in HEADS_KEYS}
    
    def to(self, device):
        if self.obs.__class__.__name__ == 'dict':
            for key in self.obs:
                self.obs[key] = self.obs[key].to(device)
        else:
            self.obs = self.obs.to(device)

        self.rewards = self.rewards.to(device)
        self.value_preds = self.value_preds.to(device)
        self.returns = self.returns.to(device)
        self.action_log_probs = {k: self.action_log_probs[k].to(device) for k in self.action_log_probs}
        self.actions = {k: self.actions[k].to(device) for k in self.actions}
        self.masks = self.masks.to(device)

    def insert(self, obs, actions, action_log_probs, value_preds, rewards, masks):
        self.copy_obs(obs, self.step + 1)
        for k in actions:
            self.actions[k][self.step].copy_(actions[k])
            self.action_log_probs[k][self.step].copy_(action_log_probs[k])
        self.value_preds[self.step].copy_(value_preds)
        self.rewards[self.step].copy_(rewards)
        self.masks[self.step + 1].copy_(masks)
        
        self.step = (self.step + 1) % self.num_steps
    
    def feed_forward_generator(self, advantages, num_mini_batch=None, mini_batch_size=None):
        num_steps, num_processes = self.rewards.size()[0:2]
        batch_size = num_processes * num_steps

        if mini_batch_size is None:
            assert batch_size >= num_mini_batch, (
                "PPO requires the number of processes ({}) "
                "* number of steps ({}) = {} "
                "to be greater than or equal to the number of PPO mini batches ({})."
                "".format(num_processes, num_steps, num_processes * num_steps,
                          num_mini_batch))
            mini_batch_size = batch_size // num_mini_batch
        sampler = BatchSampler(
            SubsetRandomSampler(range(batch_size)),
            mini_batch_size,
            drop_last=True)
        for indices in sampler:
            if self.obs.__class__.__name__ == 'dict':
                obs_batch = {key: self.obs[key][:-1].view(-1, *self.obs[key].size()[2:])[indices] for key in self.obs}
            else:
                obs_batch = self.obs[:-1].view(-1, *self.obs.size()[2:])[indices]
            
            actions_batch = {k: self.actions[k].view(-1, self.actions[k].size(-1))[indices] for k in self.actions}
            #actions_batch = self.actions.view(-1,
            #                                  self.actions.size(-1))[indices]
            
            value_preds_batch = self.value_preds[:-1].view(-1, 1)[indices]
            return_batch = self.returns[:-1].view(-1, 1)[indices]
            masks_batch = self.masks[:-1].view(-1, 1)[indices]
            
            old_action_log_probs_batch = {k: self.action_log_probs[k].view(-1, 1)[indices] for k in self.action_log_probs}
            #old_action_log_probs_batch = self.action_log_probs.view(-1,
            #                                                        1)[indices]
            if advantages is None:
                adv_targ = None
            else:
                adv_targ = advantages.view(-1, 1)[indices]

            yield obs_batch, actions_batch, \
                value_preds_batch, return_batch, masks_batch, old_action_log_probs_batch, adv_targ

In [13]:
hidden_size = NH
policy = ACN(envs[0].action_space, encoder, encoder, hidden_size, hidden_size)

agent_conf = {
        "clip_param": 0.2,
        "ppo_epoch": 4,
        "num_mini_batch": 4,
        "value_loss_coef": 0.5,
        "entropy_coef": 0.01,
        "lr": 0.0002,
        "eps": 1e-5,
        "max_grad_norm": 0.5,
}


agent = MultiheadPPO(policy, **agent_conf)
agent.to(device)

In [14]:
from tqdm import trange
import time
from collections import deque
import torch
import numpy as np


def update_linear_schedule(optimizer, epoch, total_num_epochs, initial_lr):
    """Decreases the learning rate linearly"""
    lr = initial_lr - (initial_lr * (epoch / float(total_num_epochs)))
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

def get_action_mtx():
    return [[None for _ in range(n_instances)] for _ in range(num_task)]

def get_obs_i_dict(obs, i):
    ob = {k: obs[k][i] for k in obs}
    return ob

def collect_actions(actions, agent_envs):
    action_mtx = get_action_mtx()
    for i, env_num in enumerate(agent_envs):
        action_mtx[env_num][i] = actions[i].item()
    return action_mtx
    
def collect_obs(obs, rewards, dones, infos, agent_envs):
    new_agent_envs = list()
    multi_obs, multi_reward, multi_done, multi_infos = list(), list(), list(), list()
    for i, agent_env in enumerate(agent_envs):
        multi_reward.append(rewards[agent_env][i])
        multi_infos.append(infos[agent_env][i])
        
        multi_done.append(dones[agent_env][i])
        if multi_done[-1]:
            new_agent_envs.append(np.random.randint(num_task))
        else:
            new_agent_envs.append(agent_env)
        multi_obs.append(get_obs_i_dict(obs[new_agent_envs[-1]], i))
    
    multi_obs = {k: torch.stack([ob[k] for ob in multi_obs], dim=0) for k in multi_obs[0]}    
    return multi_obs, multi_reward, multi_done, multi_infos, new_agent_envs

def perform_actions(envs, action_mtx, agent_envs):
    global jjjj
    obs, rewards, dones, infos = list(), list(), list(), list()
    for i, env_actions in enumerate(action_mtx):
        ob, reward, done, info = envs[i].step(env_actions)
        obs.append(ob); rewards.append(reward), dones.append(done), infos.append(info)
    
    obs, rewards, dones, infos, agent_envs = collect_obs(obs, rewards, dones, infos, agent_envs)
    return obs, torch.cat(rewards).view(-1, 1), torch.cat(dones), infos, agent_envs

def reset(envs, agent_envs):
    all_obs, obs = list(), list()
    for i in range(num_task):
        all_obs.append(envs[i].reset())
    for i, agent_env in enumerate(agent_envs):
        obs.append(get_obs_i_dict(all_obs[agent_env], i))
    obs = {k: torch.stack([ob[k] for ob in obs], dim=0) for k in obs[0]}
    return obs

def train_ppo(envs, agent, conf):
    """
    Runs a series of episode and collect statistics
    """
    env = envs[0]
    
    best_mean, best_median = -float('inf'), -float('inf')
    rollouts = MultiheadRolloutStorage(
        conf['training.n_steps'], conf['training.n_processes'], env.observation_space, env.action_space
    )
    
    agent_envs = np.random.randint(0, num_task, n_instances)
    obs = reset(envs, agent_envs)
    
    rollouts.set_first_obs(obs)
    rollouts.to(conf['agent.device'])

    start = time.time()
    num_updates = int(conf['training.n_env_steps'] // conf['training.n_steps'] // conf['training.n_processes'])

    episode_rewards = deque(maxlen=20)
    exploration_rewards = deque(maxlen=conf["training.n_steps"]*n_instances)
    episode_steps = deque(maxlen=20)

    for j in trange(num_updates):
        update_linear_schedule(agent.optimizer, j, num_updates, conf['agent.lr'])

        for step in range(conf['training.n_steps']):
            # Sample actions
            obs = {k: v.to(conf['agent.device']) for k, v in obs.items()}
            value, action, action_log_prob = agent.act(obs)
            
            master_action = action["master"]
            dom_action = action["dom"]
            token_action = action["token"]
            true_action = torch.where(master_action==0, dom_action, token_action + DOM_MAX_VERTICES)
            
            action_mtx = collect_actions(true_action, agent_envs)
            obs, reward, done, infos, agent_envs = perform_actions(envs, action_mtx, agent_envs)
            
            for info in infos:
                exploration_rewards.append(info["exploration_reward"])
                if 'episode' in info.keys():
                    episode_rewards.append(info['episode']['r'])
                    episode_steps.append(info['episode']['steps'])

            # If done then clean the history of observations.
            masks = torch.FloatTensor([[0.0] if done_ else [1.0] for done_ in done])
            rollouts.insert(obs, action, action_log_prob, value, reward, masks)

        next_value = agent.get_value(rollouts.get_last_obs())
        rollouts.compute_returns(next_value, conf['agent.gamma'], conf['agent.gae_lambda'])
        
        value_loss, action_loss, dist_entropy = agent.update(rollouts)

        if j % conf['training.verbose'] == 0 and len(episode_rewards) > 1:
            total_num_steps = (j + 1) * conf['training.n_processes'] * conf['training.n_steps']
            end = time.time()
            print(f'Updates {j}, '
                  f'num timesteps {total_num_steps}, '
                  f'FPS {int(total_num_steps / (end - start))} \n'
                  f'Last {len(episode_rewards)} training episodes: '
                  f'mean/median reward {np.mean(episode_rewards):.2f}/{np.median(episode_rewards):.2f}, '
                  f'min/max reward {np.min(episode_rewards):.2f}/{np.max(episode_rewards):.2f}\n'
                  f'dist_entropy {dist_entropy:.2f}, '
                  f'value_loss {value_loss:.2f}, '
                  f'action_loss {action_loss:.2f}, '
                  f'explor_rew {np.mean(exploration_rewards):.6f} '
                  f'mean_episode_steps {np.mean(episode_steps):.2f}')
            with open(conf['outputs.path']+".txt", 'a') as f:
                f.write(f"%d;%.4f;%.4f;%.6f;%.4f;%.4f\n" % \
                        (total_num_steps, np.mean(episode_rewards), \
                        np.median(episode_rewards), np.mean(exploration_rewards), \
                        dist_entropy, np.mean(episode_steps)))
            
        if j % 50 == 0:
            torch.save(agent, conf['outputs.path']+"_"+str(j))
        
        if np.mean(episode_rewards) >= best_mean:
            torch.save(agent, conf['outputs.path']+"_best_mean")
            best_mean = np.mean(episode_rewards)
        
        if np.median(episode_rewards) >= best_median:
            torch.save(agent, conf['outputs.path']+"_best_median")
            best_median = np.median(episode_rewards)

In [15]:
train_conf = {
    "agent.lr": 0.0002,
    "agent.device": device,
    "agent.gamma": 0.99,
    "agent.gae_lambda": 0.95,
    "training.n_env_steps": 8000000,
    "training.n_steps": 100,
    "training.n_processes": n_instances,
    "training.verbose": 1,
    "outputs.path": "./multitask/multitask_flat_ln"
}

In [ ]:
train_ppo(envs, agent, train_conf)

  0%|                                                                                                                                                              | 0/20000 [00:00<?, ?it/s]

Updates 0, num timesteps 400, FPS 7 
Last 20 training episodes: mean/median reward -1.04/-1.05, min/max reward -1.05/-0.95
dist_entropy 4.47, value_loss 0.68, action_loss 0.12, explor_rew -0.007750 mean_episode_steps 31.25


  0%|                                                                                                                                                  | 1/20000 [00:56<312:28:31, 56.25s/it]

Updates 1, num timesteps 800, FPS 7 
Last 20 training episodes: mean/median reward -0.92/-1.05, min/max reward -1.05/0.55
dist_entropy 4.53, value_loss 0.21, action_loss -0.01, explor_rew -0.015750 mean_episode_steps 31.15


  0%|                                                                                                                                                  | 2/20000 [01:51<309:57:35, 55.80s/it]

...

  8%|██████████▋                                                                                                                                 | 1528/20000 [24:34:22<285:37:27, 55.67s/it]

Updates 1528, num timesteps 611600, FPS 6 
Last 20 training episodes: mean/median reward 0.43/0.55, min/max reward -0.04/0.55
dist_entropy 2.66, value_loss 0.01, action_loss -0.01, explor_rew 0.031250 mean_episode_steps 19.45


  8%|██████████▋                                                                                                                                 | 1529/20000 [24:35:18<286:23:14, 55.82s/it]

Updates 1529, num timesteps 612000, FPS 6 
Last 20 training episodes: mean/median reward 0.39/0.55, min/max reward -0.04/0.55
dist_entropy 2.69, value_loss 0.00, action_loss -0.01, explor_rew 0.030000 mean_episode_steps 20.80


  8%|██████████▋                                                                                                                                 | 1530/20000 [24:36:14<286:30:22, 55.84s/it]

In [ ]:
for env in envs:
    env.close()